In [1]:
!pip install tensorboardX
!pip install ml_collections
!pip install monai
!pip install torchio
!pip install tensorflow===2.15.0
!git clone --branch training https://github.com/lesnikowka/TRUNet.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing install

In [2]:
%autosave 60

import os
import sys
import random
import numpy as np
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv3D
from keras.layers import Conv3DTranspose
from keras.layers import MaxPool3D
from keras.layers import UpSampling3D
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import MaxPool2D
from keras.layers import UpSampling2D
from keras.layers import Concatenate
from keras.layers import Flatten
from google.colab import drive
import cv2
import torchio as tio
import torch
import gc
import keras.backend as K
from datetime import datetime
from google.colab import files
import zipfile
import random
from skimage.transform import resize

Autosaving every 60 seconds


In [3]:
import os
print(os.getcwd())
import sys
sys.path.append('/content/TRUNet')

/content


In [4]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
archive = "2048cropped.zip"
!cp -r drive/MyDrive/arrays/$archive /content/sample_data/$archive
zipfile.ZipFile("/content/sample_data/" + archive, 'r').extractall("/content/sample_data")

In [6]:
NUM_CLASSES = 8
IMAGE_SIZE = 96

In [15]:
import argparse
import ml_collections
import os
from glob import glob
import torch
import numpy as np
from monai.losses import DiceCELoss
from monai.metrics import DiceMetric
from TRUNet.TRUNet_network.model.ViT import VisionTransformer3d as TransUNet3d
from datetime import datetime
from TRUNet_network.trunet_train import trainer
from torchvision import transforms
from TRUNet_network.augmentations import RandomGenerator3d_zoom, Reshape3d_zoom

now = datetime.now()

def TransUNet_configs(img_size):
    configs_trunet = ml_collections.ConfigDict()

    configs_trunet.resnet = ml_collections.ConfigDict()
    configs_trunet.resnet.num_layers = (3, 4, 9)
    configs_trunet.resnet.width_factor = 1
    configs_trunet.transformer_mlp_dim = 3072
    configs_trunet.transformer_num_heads = 12
    configs_trunet.transformer_num_layers = 12
    configs_trunet.transformer_attention_dropout_rate = 0.0
    configs_trunet.transformer_dropout_rate = 0.1
    configs_trunet.classifier = 'seg'
    configs_trunet.decoder_channels = (256, 128, 64, 16)
    configs_trunet.n_classes = 7
    configs_trunet.n_skip = 3
    configs_trunet.skip_channels = [512, 256, 64, 16]
    configs_trunet.patches = ml_collections.ConfigDict()
    configs_trunet.patches.grid = None

    configs_trunet.hidden_size = 768
    configs_trunet.patches.size = 16

    configs_trunet.patch_size = configs_trunet.patches.size  # (results in 14 by 14 grid of patches for input size 224)

    configs_trunet.patches.grid = (
        int(img_size / configs_trunet.patches.size), int(img_size / configs_trunet.patches.size),
        int(img_size / configs_trunet.patches.size))
    configs_trunet.hybrid = True

    return configs_trunet

class fetch_dataset:
    def __init__(self, base_dir, first, last):
        self.base_dir = base_dir
        self.images_dir = os.path.join(self.base_dir, "images")
        self.masks_dir = os.path.join(self.base_dir, "masks")
        self.first = first
        self.last = last

    def __len__(self):
        return self.last - self.first

    def __getitem__(self, idx):
        real_index = idx + self.first

        image_dir = os.path.join(self.images_dir, str(real_index) + ".npy")
        mask_dir = os.path.join(self.masks_dir, str(real_index) + ".npy")

        image = np.load(image_dir)
        mask = np.load(mask_dir)

        image = image[..., 0]
        mask = mask[..., 0]

        image = resize(image, (IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE), order=0, anti_aliasing=False)
        mask = resize(mask, (IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE), order=0, anti_aliasing=False)

        print(image.shape, mask.shape)

        sample = {'image': image, 'label': mask, 'case_name': str(idx)}

        return sample


if __name__ == "__main__":

    args = ml_collections.ConfigDict()
    args.max_epochs = 2
    args.save_path = "save_path"
    args.root_path = "sample_data"
    args.num_classes = NUM_CLASSES
    args.batch_size = 1
    args.seed = 42
    args.base_lr = 0.01

    # Define model
    config_net = TransUNet_configs(IMAGE_SIZE)
    config_net.n_classes = args.num_classes
    model = TransUNet3d(config_net, img_size=IMAGE_SIZE, zero_head=False, vis=False)

    config = {'ds_val': fetch_dataset(base_dir=os.path.join(args.root_path), first=0, last=5),
              'ds_train': fetch_dataset(base_dir=os.path.join(args.root_path), first=19, last=20),
              'loss_function': DiceCELoss(include_background=False, to_onehot_y=True, softmax=True),
              'metric': DiceMetric(include_background=False, reduction="mean"),
              'optimizer': torch.optim.Adam(model.parameters(), args.base_lr),
              'save_interval': 50}

    trainer(args, config, model, args.save_path)

(96, 96, 96) (96, 96, 96)
(96, 96, 96) (96, 96, 96)
(96, 96, 96) (96, 96, 96)


ValueError: not enough values to unpack (expected 5, got 4)